# Minimal Data Salesforce Data Set
> get pertinent data for contacts 4708 119430 119431

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from doubledot import *

In [139]:
from nbdev.showdoc import *
import requests
import json
import jmespath as jp
import re
from time import sleep
from fastcore.basics import patch
import fileinput
import pandas as pd
import os
import io
from fastcore.test import test_eq
import glob
import time
import random
from pyisemail import is_email

In [4]:
atms = ATMS_api()


Directory 'atms_download' already exists.
my id is mlk31g16


In [38]:
# used this to find members with 4708 contactId ( yes, there was an easier way but not faster given time and brain capacity)
atms.retrieve_and_clean('memberships', since_date='1995-01-01', max_rows=20000)

download dir is:  /Users/josephmann/Documents/Github/doubledot/atms_download/mlk31g16
resp_d = self.get_telus_data(memberships,offset=0, count= 1000, since_date=1995-01-01)
ATMS_api.get_telus_data: since_date is 1995-01-01
http://crm-api-telus.atmsplus.com/api/memberships/lastupdate?count=1000&offset=0&updateDate=1995-01-01
resp_d.keys():  dict_keys(['response', 'done'])
done: False, resp_d.json() : <Response [200]>
resp_d = self.get_telus_data(memberships,offset=1000, count= 1000, since_date=1995-01-01)
ATMS_api.get_telus_data: since_date is 1995-01-01
http://crm-api-telus.atmsplus.com/api/memberships/lastupdate?count=1000&offset=1000&updateDate=1995-01-01
resp_d.keys():  dict_keys(['response', 'done'])
done: False, resp_d.json() : <Response [200]>
resp_d = self.get_telus_data(memberships,offset=2000, count= 1000, since_date=1995-01-01)
ATMS_api.get_telus_data: since_date is 1995-01-01
http://crm-api-telus.atmsplus.com/api/memberships/lastupdate?count=1000&offset=2000&updateDate=1995-

In [41]:
_d = atms.obj_d['memberships']


,membershipMemberId,membershipTermKey,cardNumber,membershipNumber,cardStatus,contactKey,displayName
0,30001,12179,1,00325701,Active,77975,"Dnetto, William"
1,22320,12179,3,00325703,Active,77977,"Dnetto, Justin"
2,22321,12179,4,00325704,Active,77978,"Dnetto, Kevin"
3,26813,12179,2,00325702,Active,77976,"Dnetto, Jannette"
4,26814,12180,2,00325802,Active,77980,"Tonks, Debbie"


## this is from mlk31g16

the task now... 
gets contacts from contact id list
get membership from 

In [1]:
atms.obj_d['contacts']


NameError: name 'atms' is not defined

In [47]:

_l = jp.search('[]', _d)
memberships_df = pd.DataFrame(_l)[['membershipId',	'memberSince',	'updateDate'	]]
memberships_df.head()

,membershipId,memberSince,updateDate
0,3257,1999-04-25T00:00:00,2001-06-18T12:07:00.92
1,3258,1999-04-25T00:00:00,2001-06-18T12:07:00.92
2,3259,1999-04-25T00:00:00,2001-06-18T12:07:00.92
3,3260,1999-04-25T00:00:00,2001-06-18T12:07:00.92
4,3262,1999-04-28T00:00:00,2001-06-18T12:07:00.92


In [49]:
l = [	'membershipTermId',	'membershipKey',	'effectiveDate',	'expiryDate',	'membershipType',	'upgradeFromTermKey',	'giftMembership',	'refunded',	'saleDetailKey', 'itemKey']
_l = jp.search('[].membershipTerms[]', _d)
membershipTerms_df = pd.DataFrame(_l)[l]
membershipTerms_df.head()

,membershipTermId,membershipKey,effectiveDate,expiryDate,membershipType,upgradeFromTermKey,giftMembership,refunded,saleDetailKey,itemKey
0,12179,3257,1999-04-25T00:00:00,2000-04-30T23:59:59,Family Membership,NaN,False,False,NaN,7
1,12180,3258,1999-04-25T00:00:00,2000-04-30T23:59:59,Family Membership,NaN,False,False,NaN,7
2,12181,3259,1999-04-25T00:00:00,2000-04-30T23:59:59,Adult Membership,NaN,False,False,NaN,8
3,12182,3260,1999-04-25T00:00:00,2000-04-30T23:59:59,Adult Membership,NaN,False,False,NaN,8
4,11803,3262,1999-04-28T00:00:00,2000-04-30T23:59:59,Family Membership,NaN,False,False,NaN,7


In [55]:
_df = memberships_df.merge(membershipTerms_df, left_on='membershipId', right_on='membershipKey', how='inner')
total_members_df = _df.merge(members_df, left_on='membershipTermId', right_on='membershipTermKey')

In [53]:

_l = jp.search('[].membershipTerms[].membershipMembers[]', _d)
members_df = pd.DataFrame(_l)
members_df.head()

,membershipMemberId,membershipTermKey,cardNumber,membershipNumber,cardStatus,contactKey,displayName
0,30001,12179,1,00325701,Active,77975,"Dnetto, William"
1,22320,12179,3,00325703,Active,77977,"Dnetto, Justin"
2,22321,12179,4,00325704,Active,77978,"Dnetto, Kevin"
3,26813,12179,2,00325702,Active,77976,"Dnetto, Jannette"
4,26814,12180,2,00325802,Active,77980,"Tonks, Debbie"


In [58]:
pelton_df = total_members_df.loc[ total_members_df.displayName.str.contains('Pelton'),:]


In [25]:
_df.loc[_df.contactKey == 4708, :]

,membershipMemberId,membershipTermKey,cardNumber,membershipNumber,cardStatus,contactKey,displayName


## Need to retrieve sales data for the contact ids above. or for the salesdetails.

In [60]:
pelton_df.contactKey.unique()

array([  4708, 119430, 119431, 144164, 144165, 144166, 144167])

## Get contact records from contact ids

In [3]:
pelton_ids = [ 4708, 119430, 119431, 144164,144165, 144166, 144167 ]

In [153]:
l = []
for contact_id in pelton_df.contactKey.unique():
    print(contact_id)
    r = atms.get_telus_data('contacts', contact_id=contact_id)
    data_d = r['response'].json()
    print(data_d)
    l.append(data_d)
    sleep(1)

4708
http://crm-api-telus.atmsplus.com/api/contacts/4708
{'contactId': 4708, 'contactType': 'Individual', 'username': None, 'createdDate': '2004-05-05T13:53:28.42', 'updateDate': '2023-01-31T12:44:09.343', 'emailOptIn': 'Y', 'mailOptIn': 'Y', 'phoneOptIn': 'N', 'contactIndividualKey': 3, 'firstName': 'Donalda', 'lastName': 'Pelton', 'middleName': None, 'birthDate': '0001-01-01T00:00:00', 'contactOrganizationKey': 0, 'organizationName': None, 'addresses': [{'addressId': 35859, 'contactKey': 4708, 'addressType': 'Home', 'line1': '17607 93 Street NW', 'line2': None, 'line3': None, 'country': 'Canada', 'province': 'AB', 'county': None, 'city': 'Edmonton', 'postalZipCode': 'T5Z 2A6'}], 'alerts': [], 'contactTypes': [], 'emails': [{'emailId': 53622, 'contactKey': 4708, 'emailType': 'E-Mail', 'address': 'ddpelton@icloud.com'}], 'faxes': [], 'phones': [{'phoneId': 9413, 'contactKey': 4708, 'phoneType': 'Home', 'phoneNumber': '7804768945', 'extension': None}, {'phoneId': 110478, 'contactKey': 4

In [156]:
atms.obj_d['contacts'] = l

In [162]:

# sf.delete_sf_objects('Contact')
assert 'contacts' in atms.obj_d.keys()
sf.execute_job('Contact', 'insert')

execute_job
process_contacts
Salesforce: Writing 7 'Contact' objects to /Users/josephmann/Documents/Github/doubledot/sf_download/Contact.csv
Created job 7508Y00000nGxgPQAS for Contact with operation insert
Uploading job 7508Y00000nGxgPQAS of object Contact
job status: InProgress
Failed results:
"sf__Id"	"sf__Error"	LastName	MailingPostalCode	MailingCity	MailingStreet	MailingCountry	Phone	Email	External_Id__c



## Get `membership` data for contact_ids

In [159]:
## this works ..
first = True

with open('atms_sales/membership.json', 'w') as f:
    for contactId in pelton_df.contactKey.unique():
        print(contactId)
        r = atms.get_telus_data('memberships', contact_id=contactId)
        
        for line in r['response'].json():
            if first:
                f.write('[\n')
                first = False
            else:
                f.write(',\n')
            f.write(json.dumps(line))
        sleep(1)
    f.write('\n]')

4708
http://crm-api-telus.atmsplus.com/api/memberships/contact/4708
119430
http://crm-api-telus.atmsplus.com/api/memberships/contact/119430
119431
http://crm-api-telus.atmsplus.com/api/memberships/contact/119431
144164
http://crm-api-telus.atmsplus.com/api/memberships/contact/144164
144165
http://crm-api-telus.atmsplus.com/api/memberships/contact/144165
144166
http://crm-api-telus.atmsplus.com/api/memberships/contact/144166
144167
http://crm-api-telus.atmsplus.com/api/memberships/contact/144167


In [161]:
_d = json.load(open('atms_sales/membership.json'))
atms.obj_d['memberships'] = _d

In [163]:
assert 'memberships' in atms.obj_d.keys()
sf.delete_sf_objects('Membership__c')
sf.execute_job('Membership__c', 'insert')

sf.delete_sf_objects('MembershipTerm__c')
sf.execute_job('MembershipTerm__c', 'insert')

sf.delete_sf_objects('MembershipMember__c')
sf.execute_job('MembershipMember__c', 'insert')

Deleting Membership__c objects from Salesforce
Retrieving Object Ids for Membership__c from Salesforce
total number of objects =  0
In Salesforce.delete_sf_objects: Deleting 0 Membership__c objects using /Users/josephmann/Documents/Github/doubledot/sf_download/Membership__c.csv
execute_job
Created job 7508Y00000nGyCaQAK for Membership__c with operation delete
Uploading job 7508Y00000nGyCaQAK of object Membership__c
job status: JobComplete
Failed results:
"sf__Id"	"sf__Error"	Id

execute_job
Processing memberships data
Salesforce: Writing 11 memberships objects to /Users/josephmann/Documents/Github/doubledot/sf_download/Membership__c.csv
Salesforce: Writing 56 membership_terms objects to /Users/josephmann/Documents/Github/doubledot/sf_download/MembershipTerm__c.csv
Salesforce: Writing 108 membership_members objects to /Users/josephmann/Documents/Github/doubledot/sf_download/MembershipMember__c.csv
Created job 7508Y00000nGyDsQAK for Membership__c with operation insert
Uploading job 7508Y

In [2]:
pelton_df.contactKey.unique()

NameError: name 'pelton_df' is not defined

## get Sales data for conatactIds in pelton_df and write to json file

In [68]:
## this works ..
first = True

with open('atms_sales/sales2.json', 'w') as f:
    for contactId in pelton_df.contactKey.unique():
        print(contactId)
        r = atms.get_telus_data('sales', contact_id=contactId)
        
        for line in r['response'].json():
            if first:
                f.write('[\n')
                first = False
            else:
                f.write(',\n')
            f.write(json.dumps(line))
        sleep(1)
    f.write('\n]')

4708
http://crm-api-telus.atmsplus.com/api/sales/contact/4708
119430
http://crm-api-telus.atmsplus.com/api/sales/contact/119430
119431
http://crm-api-telus.atmsplus.com/api/sales/contact/119431
144164
http://crm-api-telus.atmsplus.com/api/sales/contact/144164
144165
http://crm-api-telus.atmsplus.com/api/sales/contact/144165
144166
http://crm-api-telus.atmsplus.com/api/sales/contact/144166
144167
http://crm-api-telus.atmsplus.com/api/sales/contact/144167


## Parse JSON to dictionary

In [69]:
# worked after I stitched together the lists..
w = json.load(open('atms_sales/sales2.json', 'r'))

## Put data manually into ATMS object

In [71]:
# hacky but it works
atms.obj_d['sales'] = w

In [87]:
atms

In [147]:
sf = Salesforce()
sf.atms = atms



Directory 'atms_download' already exists.


In [148]:
sf.delete_sf_objects('Ticket__c')
sf.execute_job('Ticket__c', 'insert')

Deleting Ticket__c objects from Salesforce
Retrieving Object Ids for Ticket__c from Salesforce
total number of objects =  200
In Salesforce.delete_sf_objects: Deleting 200 Ticket__c objects using /Users/josephmann/Documents/Github/doubledot/sf_download/Ticket__c.csv
execute_job
Created job 7508Y00000nGxG6QAK for Ticket__c with operation delete
Uploading job 7508Y00000nGxG6QAK of object Ticket__c
job status: InProgress
Failed results:
"sf__Id"	"sf__Error"	Id

execute_job
Salesforce: Writing 210 'Ticket' objects to /Users/josephmann/Documents/Github/doubledot/sf_download/Ticket__c.csv
Created job 7508Y00000nGxGBQA0 for Ticket__c with operation insert
Uploading job 7508Y00000nGxGBQA0 of object Ticket__c
job status: InProgress
waiting for job to complete, try 0, status: JobComplete
Failed results:
"sf__Id"	"sf__Error"	ticketKey__c	saleKey__r.saleKey__c	saleDetailKey__r.saleDetailId__c	itemDescription__c	ticketDisplay__c



In [ ]:
s = sf.failed_results().text
pd.read_csv( io.StringIO(s), sep='\t' )

In [142]:
sf.delete_sf_objects('SaleDetail__c')
sf.execute_job('SaleDetail__c', 'insert')

Deleting SaleDetail__c objects from Salesforce
Retrieving Object Ids for SaleDetail__c from Salesforce
total number of objects =  0
In Salesforce.delete_sf_objects: Deleting 0 SaleDetail__c objects using /Users/josephmann/Documents/Github/doubledot/sf_download/SaleDetail__c.csv
execute_job
Created job 7508Y00000nGxDbQAK for SaleDetail__c with operation delete
Uploading job 7508Y00000nGxDbQAK of object SaleDetail__c
job status: InProgress
waiting for job to complete, try 0, status: InProgress
waiting for job to complete, try 1, status: InProgress
Failed results:
"sf__Id"	"sf__Error"	Id

execute_job
Salesforce: Writing 319 'SaleDetail' objects to /Users/josephmann/Documents/Github/doubledot/sf_download/SaleDetail__c.csv
Created job 7508Y00000nGxDgQAK for SaleDetail__c with operation insert
Uploading job 7508Y00000nGxDgQAK of object SaleDetail__c
job status: InProgress
waiting for job to complete, try 0, status: InProgress
waiting for job to complete, try 1, status: InProgress
Failed resu

In [140]:
s = sf.failed_results().text
pd.read_csv( io.StringIO(s), sep='\t' )

,sf__Id,sf__Error,saleDetailId__c,itemKey__c,scheduleKey__c,rateKey__c,categoryKey__c,itemCategory__c,pricingPriceKey__c,itemPrice__c,itemTotal__c,couponTotal__c,discountTotal__c,total__c,revenueDate__c,refundReason__c,refundReasonKey__c,systemPriceOverride__c,MembershipTermKey__r.MembershipTermId__c,saleId__c
0,NaN,MALFORMED_ID:saleId: id value of incorrect typ...,199.0,30.0,119.0,1.0,2.0,NaN,163.0,115.00,115.00,NaN,23.0,92.00,2001-07-03,NaN,NaN,N,NaN,75
1,NaN,MALFORMED_ID:saleId: id value of incorrect typ...,201.0,31.0,130.0,1.0,2.0,NaN,173.0,115.00,115.00,NaN,23.0,92.00,2001-08-07,NaN,NaN,N,NaN,75
2,NaN,MALFORMED_ID:saleId: id value of incorrect typ...,14763.0,2.0,852.0,1.0,1.0,NaN,10.0,8.45,2315.30,2315.30,NaN,NaN,2001-07-24,NaN,NaN,N,NaN,6200
3,NaN,MALFORMED_ID:saleId: id value of incorrect typ...,14764.0,4.0,3917.0,1.0,1.0,NaN,10.0,8.45,2315.30,2315.30,NaN,NaN,2001-07-24,NaN,NaN,N,NaN,6200
4,NaN,MALFORMED_ID:saleId: id value of incorrect typ...,71113.0,166.0,6109.0,1.0,1.0,NaN,10.0,8.45,1478.75,1478.75,NaN,NaN,2001-10-05,NaN,NaN,N,NaN,28717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,NaN,MALFORMED_ID:saleId: id value of incorrect typ...,3435312.0,1412.0,340806.0,1.0,1.0,NaN,13921.0,26.50,-53.00,NaN,NaN,-55.65,2013-03-26,NaN,1090.0,N,NaN,1019680
315,NaN,MALFORMED_ID:saleId: id value of incorrect typ...,3435313.0,1412.0,340805.0,1.0,1.0,NaN,13921.0,26.50,53.00,NaN,NaN,55.65,2013-03-26,NaN,NaN,N,NaN,1019680
316,NaN,MALFORMED_ID:saleId: id value of incorrect typ...,3435314.0,1513.0,334619.0,1.0,1.0,NaN,13394.0,13.95,27.90,27.90,NaN,NaN,2013-03-26,NaN,NaN,N,NaN,1019680
317,NaN,MALFORMED_ID:saleId: id value of incorrect typ...,760379.0,7.0,NaN,3.0,NaN,NaN,207.0,105.00,105.00,NaN,NaN,123.05,2005-08-19,NaN,NaN,N,NaN,310928


In [117]:
sf.execute_job('Sale__c', 'insert')

execute_job
Salesforce: Writing 82 'Sales' objects to /Users/josephmann/Documents/Github/doubledot/sf_download/Sale__c.csv
Created job 7508Y00000nGwidQAC for Sale__c with operation insert
Uploading job 7508Y00000nGwidQAC of object Sale__c
job status: InProgress
Failed results:
"sf__Id"	"sf__Error"	saleKey__c	saleAmount__c	paymentAmount__c	saleDate__c	active__c	terminalKey__c	eventDate__c	booking_bookingId__c	booking_bookingContactKey__c	booking_bookingContactType__c	booking_contactKey__c	booking_contactIndividualKey__c	booking_contactOrganizationKey__c	booking_displayName__c	booking_firstName__c	booking_lastName__c	booking_email__c	booking_phone__c
""	"DUPLICATE_VALUE:duplicate value found: saleKey__c duplicates value on record with id: a058Y00001B5QTa:--"	"75"	"184.0"	"184.0"	"2001-06-18"	"true"	"2.0"	""	""	""	""	""	""	""	""	""	""	""	""
""	"DUPLICATE_VALUE:duplicate value found: saleKey__c duplicates value on record with id: a058Y00001B5QTb:--"	"6200"	"0.0"	"0.0"	"2001-07-16"	"true"	"

In [111]:
import io
s = sf.failed_results().text
pd.read_csv( io.StringIO(s), sep='\t' )

,sf__Id,sf__Error,saleKey__c,saleAmount__c,paymentAmount__c,saleDate__c,active__c,terminalKey__c,booking_bookingId__c,booking_bookingContactKey__c,booking_bookingContactType__c,booking_contactKey__c,booking_contactIndividualKey__c,booking_contactOrganizationKey__c,booking_displayName__c,booking_firstName__c,booking_lastName__c,booking_email__c,booking_phone__c
0,NaN,INVALID_FIELD:Failed to deserialize field at c...,75,184.00,184.00,2001-06-18,True,2.0,,,,,,,,,,,
1,NaN,INVALID_FIELD:Failed to deserialize field at c...,6200,0.00,0.00,2001-07-16,True,11.0,,,,,,,,,,,
2,NaN,INVALID_FIELD:Failed to deserialize field at c...,28717,0.00,0.00,2001-10-02,True,3.0,,,,,,,,,,,
3,NaN,INVALID_FIELD:Failed to deserialize field at c...,34334,0.00,0.00,2001-10-27,True,11.0,,,,,,,,,,,
4,NaN,INVALID_FIELD:Failed to deserialize field at c...,35028,0.00,0.00,2001-10-31,True,15.0,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,NaN,INVALID_FIELD:Failed to deserialize field at c...,1615688,0.00,0.00,2017-12-19,True,1.0,,,,,,,,,,,
78,NaN,INVALID_FIELD:Failed to deserialize field at c...,1776435,0.00,0.00,2019-03-20,True,99.0,,,,,,,,,,,
79,NaN,INVALID_FIELD:Failed to deserialize field at c...,1019680,55.65,55.65,2013-03-24,True,15.0,,,,,,,,,,,
80,NaN,INVALID_FIELD:Failed to deserialize field at c...,310928,123.05,123.05,2005-08-19,True,4.0,,,,,,,,,,,
